In [24]:
import os, sys
import time
import datetime
import pandas as pd
import numpy as np
import math
from math import radians, cos, sin, asin, sqrt 
import random
import copy

ROOTDIR = os.path.abspath(os.path.realpath('./')) + '/Py'

sys.path.append(os.path.join(ROOTDIR, ''))

from Stamp_transition import Stamp_transition

from Dynamic_programming import Dynamic_programming


# Parameter

In [25]:
'''Basic Path'''

Daily_path='./Data/Daily_Feature/'

Load_path='./Data/Processed/'

Save_path='./Data/CMMDP/'

date_str='2019-11-01'



'''
Dynamic programming for Q values

'''

'''Location list'''

Location_list=np.load(os.path.join(Load_path,'Location_list.npy'))

Location_ID_dic=np.load(os.path.join(Load_path,'Location_ID_dic.npy')).item()

Location_ID_dic_reverse=np.load(os.path.join(Load_path,'Location_ID_dic_reverse.npy')).item()

'''Location Center'''

Location_Center_dic=np.load(os.path.join(Load_path,'Location_Center_dic.npy')).item()


'''Connection Matrix and Network distance Matrix'''

Connect_matrix=np.load(os.path.join(Load_path,'Connect_matrix.npy'))

Network_Distance=np.load(os.path.join(Load_path,'Network_Distance.npy'))

'''Geometry_dic'''

Geometry_dic=np.load(os.path.join(Load_path,'Geometry_dic.npy')).item()

'''State and Action'''


stamp_transition=Stamp_transition()

Yesterday=stamp_transition.Get_Yesterday(date_str)

'''State and Action'''

State=np.load(os.path.join(Load_path,'State.npy'))

Action=np.load(os.path.join(Load_path,'Action.npy')).item()

'''Prob matrix'''

Dest_PROB_dic=np.load(os.path.join(Daily_path,'Dest_PROB_dic'+Yesterday+'.npy')).item()

Gain_dic=np.load(os.path.join(Daily_path,'Gain_dic'+Yesterday+'.npy')).item()



# Match Probability

In [26]:
def Get_prob(ratio):
        
    prob = 1- np.exp(-1*ratio)

    return prob

In [27]:
End_step=144

Request_count_dic=np.load(os.path.join(Daily_path,'Request_count_dic'+Yesterday+'.npy')).item()


'''Historical Driver data '''

Driver_data=pd.read_csv(os.path.join(Save_path,'Driver_data'+Yesterday+'.csv'))

Driver_count=Driver_data.groupby(['Location_id','step']).count()[['Driver_id']]

Driver_count['Transition']=Driver_count.index

Driver_count['Pickup_Location']=Driver_count.apply(lambda x:x['Transition'][0],axis=1)

Driver_count['Pickup_step']=Driver_count.apply(lambda x:x['Transition'][1],axis=1)

Driver_count=Driver_count.rename(index=str, columns={"Driver_id": "Driver_Sum"})

Driver_count=Driver_count.reset_index(drop=True)

Driver_count=Driver_count[['Pickup_Location','Pickup_step','Driver_Sum']]



Match_PROB_dic={state:0.0 for state in State}

for idx,row in Driver_count.iterrows():
    
    if row['Pickup_step']<End_step:
    
        state=str(int(row['Pickup_Location']))+'-'+str(int(row['Pickup_step']))

        if row['Driver_Sum']==0:

            ratio=999

            Match_PROB_dic[state]=Get_prob(ratio)

        else:

            ratio=Request_count_dic[state]/row['Driver_Sum']

            Match_PROB_dic[state]=Get_prob(ratio)
        
    
    
    
    
    



In [28]:
Dynamic_P=Dynamic_programming(Location_list,State,Action,End_step,Gain_dic,Match_PROB_dic,Dest_PROB_dic)


Dynamic_P.Q_table


{'127-0': {127: 0.0,
  128: 0.0,
  243: 87.53511340773659,
  244: 0.0,
  120: 0.0,
  153: 0.0},
 '127-1': {127: 0.0,
  128: 0.0,
  243: 41.408215595964634,
  244: 34.15199506187352,
  120: 0.0,
  153: 0.0},
 '127-2': {127: 0.0, 128: 0.0, 243: 0.0, 244: 0.0, 120: 0.0, 153: 0.0},
 '127-3': {127: 0.0, 128: 0.0, 243: 0.0, 244: 0.0, 120: 0.0, 153: 0.0},
 '127-4': {127: 0.0,
  128: 0.0,
  243: 0.0,
  244: 28.97705645643831,
  120: 0.0,
  153: 0.0},
 '127-5': {127: 0.0,
  128: 0.0,
  243: 0.0,
  244: 40.352875788030886,
  120: 0.0,
  153: 0.0},
 '127-6': {127: 0.0,
  128: 0.0,
  243: 0.0,
  244: 31.030267626203802,
  120: 0.0,
  153: 0.0},
 '127-7': {127: 0.0,
  128: 0.0,
  243: 75.97247370278954,
  244: 0.0,
  120: 0.0,
  153: 0.0},
 '127-8': {127: 0.0,
  128: 0.0,
  243: 0.0,
  244: 38.0871908794802,
  120: 0.0,
  153: 0.0},
 '127-9': {127: 0.0,
  128: 0.0,
  243: 28.995273913054305,
  244: 0.0,
  120: 0.0,
  153: 0.0},
 '127-10': {127: 0.0,
  128: 0.0,
  243: 0.0,
  244: 49.00242443729783,